In [1]:

import pydicom
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
keras = tf.keras
import boto3
import os
import pathlib
import dicom
import dicom_numpy
import PIL
import dill as pickle

/anaconda3/envs/learn-env/lib/python3.7/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
!ls

EDA-Copy1.ipynb                 data
EDA.ipynb                       data.pkl
EULA.txt                        legalcode.txt
README.md                       prediction_probabilities.csv
capstone_proposal_template.gdoc reads.csv
cq500_files.txt


In [3]:
df_labels = pd.read_csv('reads.csv')
df_labels.T

,0,1,2,3,4,5,6,7,8,9,...,481,482,483,484,485,486,487,488,489,490
name,CQ500-CT-427,CQ500-CT-181,CQ500-CT-99,CQ500-CT-47,CQ500-CT-195,CQ500-CT-357,CQ500-CT-80,CQ500-CT-2,CQ500-CT-361,CQ500-CT-412,...,CQ500-CT-22,CQ500-CT-65,CQ500-CT-125,CQ500-CT-406,CQ500-CT-251,CQ500-CT-56,CQ500-CT-438,CQ500-CT-126,CQ500-CT-348,CQ500-CT-382
Category,B2,B2,B1,B1,B1,B2,B2,B2,B2,B2,...,B2,B2,B2,B2,B1,B2,B2,B2,B2,B2
R1:ICH,1,1,0,0,0,0,1,0,0,1,...,1,1,1,1,1,1,1,1,1,1
R1:IPH,1,1,0,0,0,0,1,0,0,0,...,1,0,1,0,1,0,0,1,0,0
R1:IVH,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R1:SDH,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
R1:EDH,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
R1:SAH,0,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,0,0,1,1
R1:BleedLocation-Left,0,1,0,0,0,0,1,0,0,1,...,0,1,1,1,1,1,1,1,1,1
R1:BleedLocation-Right,1,0,0,0,0,0,1,0,0,1,...,1,0,0,1,1,0,1,0,1,1


In [4]:
ich = {'ich': [col for col in df_labels.columns if 'ICH' in col]}
iph = {'iph': [col for col in df_labels.columns if 'IPH' in col]}
ivh= {'ivh': [col for col in df_labels.columns if 'IVH' in col]}
sdh = {'sdh': [col for col in df_labels.columns if 'SDH' in col]}
edh = {'edh': [col for col in df_labels.columns if 'EDH' in col]}
sah = {'sah': [col for col in df_labels.columns if 'SAH' in col]}
bll =  {'bll': [col for col in df_labels.columns if 'Left' in col]}
blr = {'blr': [col for col in df_labels.columns if 'Right' in col]}
cb = {'cb': [col for col in df_labels.columns if 'ChronicBleed' in col]}
cfracture = {'cfracture': [col for col in df_labels.columns if 'CalvarialFracture' in col]}
ofracture = {'ofracture': [col for col in df_labels.columns if 'OtherFracture' in col]}
ma = {'ma': [col for col in df_labels.columns if 'MassEffect' in col]}
ms = {'ms': [col for col in df_labels.columns if 'MidlineShift' in col]}
cols = [ich, iph, ivh, sdh, edh, sah, bll, blr, cb, cfracture, ofracture, ma, ms]


for col in cols:

    df_labels[next(iter(col.keys()))] = df_labels[next(iter(col.values()))].apply(sum, axis=1)

In [5]:
drop_columns = [col for col in df_labels.columns if 'R1' in col or 'R2' in col or 'R3' in col]

In [6]:
df_labels = df_labels.drop(drop_columns, axis=1)

In [7]:
df_labels.T

,0,1,2,3,4,5,6,7,8,9,...,481,482,483,484,485,486,487,488,489,490
name,CQ500-CT-427,CQ500-CT-181,CQ500-CT-99,CQ500-CT-47,CQ500-CT-195,CQ500-CT-357,CQ500-CT-80,CQ500-CT-2,CQ500-CT-361,CQ500-CT-412,...,CQ500-CT-22,CQ500-CT-65,CQ500-CT-125,CQ500-CT-406,CQ500-CT-251,CQ500-CT-56,CQ500-CT-438,CQ500-CT-126,CQ500-CT-348,CQ500-CT-382
Category,B2,B2,B1,B1,B1,B2,B2,B2,B2,B2,...,B2,B2,B2,B2,B1,B2,B2,B2,B2,B2
ich,3,3,0,0,0,0,3,0,0,2,...,3,3,3,3,3,1,3,3,3,2
iph,3,1,0,0,0,0,3,0,0,0,...,3,1,3,0,2,0,0,3,1,0
ivh,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sdh,0,1,0,0,0,0,0,0,0,2,...,0,2,0,0,1,0,3,0,0,0
edh,0,2,0,0,0,0,0,0,0,0,...,0,0,2,0,1,0,1,0,0,0
sah,0,2,0,0,0,0,1,0,0,0,...,0,1,1,3,2,1,1,0,3,2
bll,2,3,0,0,0,0,3,0,0,3,...,0,3,3,2,2,1,3,3,2,1
blr,3,0,0,0,0,0,1,0,0,3,...,3,0,0,2,2,0,1,0,1,2


In [8]:
for i, row in df_labels.iterrows():
    for j, col in enumerate(df_labels.columns):

        
        if 'CQ' in str(row.iloc[j]) or 'B' in str(row.iloc[j]):
            pass
        elif int(row.iloc[j]) > 2:
            df_labels.iloc[i,j] = 1
        else:
            df_labels.iloc[i, j] = 0

In [9]:
df_labels.T

,0,1,2,3,4,5,6,7,8,9,...,481,482,483,484,485,486,487,488,489,490
name,CQ500-CT-427,CQ500-CT-181,CQ500-CT-99,CQ500-CT-47,CQ500-CT-195,CQ500-CT-357,CQ500-CT-80,CQ500-CT-2,CQ500-CT-361,CQ500-CT-412,...,CQ500-CT-22,CQ500-CT-65,CQ500-CT-125,CQ500-CT-406,CQ500-CT-251,CQ500-CT-56,CQ500-CT-438,CQ500-CT-126,CQ500-CT-348,CQ500-CT-382
Category,B2,B2,B1,B1,B1,B2,B2,B2,B2,B2,...,B2,B2,B2,B2,B1,B2,B2,B2,B2,B2
ich,1,1,0,0,0,0,1,0,0,0,...,1,1,1,1,1,0,1,1,1,0
iph,1,0,0,0,0,0,1,0,0,0,...,1,0,1,0,0,0,0,1,0,0
ivh,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sdh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
edh,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
bll,0,1,0,0,0,0,1,0,0,1,...,0,1,1,0,0,0,1,1,0,0
blr,1,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,0


In [10]:
file = df_labels['name'][0]
filename = file + ' ' + file
def extract_voxel_data(list_of_dicom_files):
    datasets = [pydicom.read_file(f) for f in list_of_dicom_files]
    try:
        voxel_ndarray, ijk_to_xyz = dicom_numpy.combine_slices(datasets)
    except dicom_numpy.DicomImportException as e:
        # invalid DICOM data
        raise
    return voxel_ndarray


def extract_data_2d(list_of_dicom_files):
    datasets = [pydicom.read_file(f) for f in list_of_dicom_files]
    
    return datasets

In [11]:
root = './data/'
filenames = []

from pathlib import Path
result = list(Path(".").rglob("*.dcm"))
result = [str(path) for path in result]

In [12]:
# data2d = {}
# current_img = None
# images = []
# for image in result:
#     if current_img == None:
#         current_img = image[:-12]
    
#     if image[:-12] == current_img:
#         images.append(image)
        
#     else:

#         try:
#             data2d[current_img] = extract_data_2d(images)
#             current_img = image[:-12]
#             images = []
#             images.append(image)
#         except:
#             continue


In [13]:
s3 = boto3.resource('s3')

In [14]:
bucket = s3.Bucket('final-project-ct-scans')

In [18]:
s3.Object('mybucket', 'data2d.pkl').put(Body=open(pickle.dumps(data2d), 'rb'))



KeyboardInterrupt: 

In [ ]:
#  with open('data_2d.pkl', 'wb') as f:

#     pickle.dump(data, f)

In [ ]:
# data = {}
# current_img = None
# images = []
# for image in result:
    
#     if current_img == None:
#         current_img = image[:-12]
    
#     if image[:-12] == current_img:
#         images.append(image)
        
#     else:

#         try:
#             data[current_img] = extract_voxel_data(images)
#             current_img = image[:-12]
#             images = []
#             images.append(image)
#         except:
#             continue


In [13]:
# with open('data.pkl', 'wb') as f:

#     pickle.dump(data, f)

In [ ]:
# with open('data.pkl', 'rb') as f:
#     data = pickle.load(f)

In [46]:
# from mpl_toolkits.mplot3d import Axes3D

# for stuff in data.items():
#     fig = plt.figure()
#     ax = fig.gca(projection='3d')
#     ax.voxels(stuff[1])
#     break

In [47]:
for img in data.items():
    n_voxels = np.prod(img[1].shape[:-1])
    print(n_voxels)
     n_trs = img[1].shape[-1]
    data = img[1].get_data()
    std_devs = []
    for vol_no in range(n_trs):
        vol = data[..., vol_no]
        std_devs.append(np.std(vol))
    plt.plot(std_devs)
    break


NameError: name 'img' is not defined